In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import FactorAnalysis
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import os

# ----------------------------------
# 🔧 텍스트 전처리 함수
# ----------------------------------
def preprocess_text(text):
    if pd.isnull(text):
        return ""

    if isinstance(text, list):
        text = ' '.join(text)

    # HTML/XML 제거
    if isinstance(text, str) and ('<' in text and '>' in text):
        text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"\$.*?\$", "", text)
    text = re.sub(r"\\\(.*?\\\)", "", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = text.lower()

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return " ".join(tokens)

# ----------------------------------
# 📦 전처리 및 벡터화
# ----------------------------------
def preprocess_and_vectorize(df, method='count', max_features=5000, data_type='journal'):
    if data_type == 'journal':
        text_columns = ['title', 'abstract', 'keywords']
    elif data_type == 'article':
        text_columns = ['title', 'content', 'keywords']
    else:
        raise ValueError("data_type은 'journal' 또는 'article' 중 하나여야 합니다.")

    for col in text_columns:
        if col not in df.columns:
            df[col] = ''
        df[f'{col}_clean'] = df[col].apply(preprocess_text)

    df['combined_text'] = df[[f'{col}_clean' for col in text_columns]].agg(' '.join, axis=1)

    if method == 'count':
        vectorizer = CountVectorizer(max_features=max_features, stop_words='english', max_df=0.90)
    elif method == 'tfidf':
        vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english', max_df=0.90)
    else:
        raise ValueError("method는 'count' 또는 'tfidf' 중 하나여야 합니다.")

    vectorized_matrix = vectorizer.fit_transform(df['combined_text'])
    return vectorizer, vectorized_matrix

# ----------------------------------
# 🔍 문서 준비 및 토큰화
# ----------------------------------
def prepare_documents(df, data_type='journal'):
    df = df.fillna('')

    if 'combined_text' not in df.columns:
        if data_type == 'journal':
            text_columns = ['title', 'abstract', 'keywords']
        elif data_type == 'article':
            text_columns = ['title', 'content', 'keywords']
        else:
            raise ValueError("data_type은 'journal' 또는 'article' 중 하나여야 합니다.")

        for col in text_columns:
            if col not in df.columns:
                df[col] = ''
        df['combined_text'] = df[text_columns].agg(' '.join, axis=1)
        df['combined_text'] = df['combined_text'].apply(preprocess_text)

    return [doc.split() for doc in df['combined_text']]

# ----------------------------------
# 📈 Coherence 점수 계산
# (여기서는 안쓰지만 남겨둠)
# ----------------------------------
def compute_coherence_scores(dictionary, corpus, texts, start, limit, step):
    scores = []
    for k in range(start, limit, step):
        lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, random_state=42, passes=10)
        cm = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
        scores.append((k, cm.get_coherence()))
    return scores

# ----------------------------------
# 🧩 토픽-문서 행렬 생성
# ----------------------------------
def extract_topic_matrix(lda_model, corpus, num_topics):
    topic_matrix = []
    for doc in corpus:
        dist = lda_model.get_document_topics(doc, minimum_probability=0)
        topic_matrix.append([prob for _, prob in sorted(dist)])
    return pd.DataFrame(topic_matrix, columns=[f"Topic_{i}" for i in range(num_topics)])

# ----------------------------------
# 📊 요인 분석
# ----------------------------------
def run_factor_analysis(topic_df, n_factors=5, max_iter=500):
    fa = FactorAnalysis(n_components=n_factors, random_state=42, max_iter=max_iter)
    factors = fa.fit_transform(topic_df)
    loadings = pd.DataFrame(fa.components_.T, index=topic_df.columns, columns=[f"Factor_{i+1}" for i in range(n_factors)])
    return pd.DataFrame(factors, columns=loadings.columns), loadings

# ----------------------------------
# 📊 요인 분석 상위 문서 5개 저장
# ----------------------------------
# ----------------------------------
# 📊 요인 분석 상위 문서 5개 저장 (500글자 제한 추가)
# ----------------------------------
def top_docs_by_factor(factor_df, docs_df, top_n=5, output_path='top_documents_by_factor.txt', data_type='journal'):
    with open(output_path, 'w', encoding='utf-8') as f:
        for factor in factor_df.columns:
            f.write(f"\n📌 상위 문서 - {factor}\n")
            f.write("="*60 + "\n")
            top_indices = factor_df[factor].nlargest(top_n).index

            for i in top_indices:
                row = docs_df.loc[i]

                f.write(f"📆 Date: {row.get('date', '')}\n")
                f.write(f"📄 Title: {row.get('title', '')}\n")

                if data_type == 'journal':
                    content = row.get('abstract', '')
                elif data_type == 'article':
                    content = row.get('content', '')
                else:
                    raise ValueError("data_type은 'journal' 또는 'article' 중 하나여야 합니다.")

                # 🔵 본문 500글자까지만 저장
                if len(content) > 500:
                    content = content[:500].rstrip() + "..."

                f.write(f"🔍 Content (500자 이내): {content}\n")
                f.write(f"🏷️ Keywords: {row.get('keywords', '')}\n")
                f.write("-"*60 + "\n")

    print(f"✅ 저장 완료 (본문 500자 제한 적용): {output_path}")

# ----------------------------------
# 📝 LDA 모델로부터 토픽별 키워드 저장
# ----------------------------------
def save_lda_topics(lda_model, num_words, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for idx, topic in lda_model.show_topics(num_topics=-1, num_words=num_words, formatted=False):
            keywords = ", ".join([word for word, _ in topic])
            f.write(f"Topic {idx}: {keywords}\n")
    print(f"✅ LDA 토픽 저장 완료: {output_path}")

# ----------------------------------
# 🎯 년도별로 LDA + Factor 분석
# ----------------------------------
def run_yearly_lda_factor_analysis(df, data_type='journal', n_topics=10, n_factors=5, vectorizer_method='tfidf', max_features=5000, output_dir='results'):
    os.makedirs(output_dir, exist_ok=True)

    years = sorted(df['date'].unique())

    for year in years:
        print(f"🔵 Processing year: {year}")

        # 해당 연도 데이터 추출
        year_df = df[df['date'] == year].reset_index(drop=True)

        # 벡터화
        vectorizer, vectorized_matrix = preprocess_and_vectorize(year_df, method=vectorizer_method, max_features=max_features, data_type=data_type)

        # Gensim LDA용 corpus 준비
        processed_docs = prepare_documents(year_df, data_type=data_type)
        dictionary = corpora.Dictionary(processed_docs)
        corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

        # LDA 모델 학습
        lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=n_topics, random_state=42, passes=10)

        # 🎯 LDA 토픽별 키워드 저장 추가
        save_lda_topics(
            lda_model=lda_model,
            num_words=10,  # 토픽당 상위 10개 단어
            output_path=f"{output_dir}/02_{data_type}_{year}_lda_topics.txt"
        )

        # 토픽-문서 행렬 생성
        topic_df = extract_topic_matrix(lda_model, corpus, n_topics)

        # Factor Analysis
        factor_df, loadings = run_factor_analysis(topic_df, n_factors=n_factors)

        # 결과 저장
        topic_df.to_csv(f"{output_dir}/02_{data_type}_{year}_topic_matrix.csv", index=False)
        factor_df.to_csv(f"{output_dir}/02_{data_type}{year}_factor_scores.csv", index=False)
        loadings.to_csv(f"{output_dir}/02_{data_type}{year}_factor_loadings.csv", index=True)

        top_docs_by_factor(
            factor_df=factor_df,
            docs_df=year_df,
            top_n=5,
            output_path=f"{output_dir}/02_{data_type}_{year}_top_docs_by_factor.txt",  # 여기 수정
            data_type=data_type
        )

        print(f"✅ Year {year} 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)")


In [2]:
df=pd.read_csv('data/01_combined_article.csv')

In [3]:
run_yearly_lda_factor_analysis(
    df,  # 👉 당신이 만든 데이터프레임
    data_type='article',  # 👉 논문이면 'journal', 뉴스 기사면 'article'
    n_topics=10,          # 👉 고정: LDA 토픽 개수
    n_factors=5,          # 👉 고정: Factor Analysis 요인 수
    vectorizer_method='tfidf',  # 👉 'count'나 'tfidf' 중 선택 ('tfidf' 추천)
    max_features=5000,     # 👉 벡터라이저 최대 단어 수 (선택사항)
    output_dir='data/result/02'   # 👉 결과 저장 폴더명
)

🔵 Processing year: 2021
✅ LDA 토픽 저장 완료: data/result/02/02_article_2021_lda_topics.txt
✅ 저장 완료 (본문 500자 제한 적용): data/result/02/02_article_2021_top_docs_by_factor.txt
✅ Year 2021 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)
🔵 Processing year: 2022
✅ LDA 토픽 저장 완료: data/result/02/02_article_2022_lda_topics.txt
✅ 저장 완료 (본문 500자 제한 적용): data/result/02/02_article_2022_top_docs_by_factor.txt
✅ Year 2022 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)
🔵 Processing year: 2023
✅ LDA 토픽 저장 완료: data/result/02/02_article_2023_lda_topics.txt


/opt/anaconda3/envs/AI_news/lib/python3.11/site-packages/sklearn/decomposition/_factor_analysis.py:296: ConvergenceWarning: FactorAnalysis did not converge. You might want to increase the number of iterations.
  warnings.warn(


✅ 저장 완료 (본문 500자 제한 적용): data/result/02/02_article_2023_top_docs_by_factor.txt
✅ Year 2023 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)
🔵 Processing year: 2024
✅ LDA 토픽 저장 완료: data/result/02/02_article_2024_lda_topics.txt


/opt/anaconda3/envs/AI_news/lib/python3.11/site-packages/sklearn/decomposition/_factor_analysis.py:296: ConvergenceWarning: FactorAnalysis did not converge. You might want to increase the number of iterations.
  warnings.warn(


✅ 저장 완료 (본문 500자 제한 적용): data/result/02/02_article_2024_top_docs_by_factor.txt
✅ Year 2024 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)
🔵 Processing year: 2025
✅ LDA 토픽 저장 완료: data/result/02/02_article_2025_lda_topics.txt
✅ 저장 완료 (본문 500자 제한 적용): data/result/02/02_article_2025_top_docs_by_factor.txt
✅ Year 2025 완료! (Topic Matrix, Factor Scores, Loadings, Top Docs 저장)


# 데이터 라벨링 작업 - 토픽모델링

## 2021년
- Google Search: Topic 0: google, company, new, also, data, year, like, search, said, user
- Facial Recognition: Topic 1: recognition, facial, system, uber, company, like, technology, driver, model, would
- Customer Features: Topic 2: also, company, new, like, user, customer, feature, say, time, million
- Startup: Topic 3: company, people, like, startup, tech, think, year, one, work, automation
- Voice Technology: Topic 4: company, voice, tesla, like, also, system, say, technology, one, time
- Platform: Topic 5: company, data, said, platform, startup, venture, customer, also, year, new
- Machine Learning: Topic 6: data, model, company, learning, machine, robot, like, time, microsoft, one
- Human-Technology Systems: Topic 7: data, company, like, research, also, human, google, work, system, say
- Social Media: Topic 8: facebook, system, data, also, company, say, people, year, one, platform
- Programming: Topic 9: codex, code, openai, food, company, people, one, video, like, say

## 2022년
- Robotics: Topic 0: robot, company, google, robotics, like, system, new, year, also, one
- Data-Driven Analysis: Topic 1: data, model, like, also, carbon, clearview, one, work, say, company
- Language Models: Topic 2: language, model, data, google, company, like, say, system, new, tool
- Startup: Topic 3: company, data, said, startup, also, like, technology, year, new, platform
- Platform: Topic 4: data, company, nvidia, said, platform, also, new, time, say, year
- Supply Chain: Topic 5: microsoft, data, image, skin, business, use, startup, supply, say, chain
- Human-AI Interaction: Topic 6: mem, say, human, machine, company, like, musk, robot, people, one
- Customer Service: Topic 7: company, startup, customer, said, data, platform, service, new, year, capital
- Generative AI: Topic 8: image, model, like, system, text, art, video, diffusion, work, make
- Human-Technology Systems: Topic 9: company, data, like, user, also, new, say, use, technology, startup

## 2023년
- EMS: Topic 0: user, data, company, also, tool, like, new, microsoft, say, use
- Chat GPT: Topic 1: openai, chatgpt, microsoft, model, gpt, copilot, company, user, like, text
- Generative AI: Topic 2: model, data, company, generative, also, said, risk, open, source, like
- Online Communities: Topic 3: think, like, thing, going, people, one, way, really, lot, get
- OpenAI Governance: Topic 4: openai, altman, microsoft, board, company, ceo, said, new, team, sam
- Google Search: Topic 5: google, new, search, like, tool, company, also, content, user, image
- Search Engine: Topic 6: google, new, feature, microsoft, bing, like, also, user, search, app
- Music Tools: Topic 7: music, company, artist, like, work, new, tool, year, one, song
- Startup: Topic 8: company, model, generative, startup, image, data, like, said, new, customer
- Voice Technology: Topic 9: voice, company, like, people, musk, one, said, time, would, say

## 2024년
- Online Communities: Topic 0: think, people, going, thing, way, like, right, really, work, get
- Startup: Topic 1: startup, data, company, model, said, year, also, new, million, generative
- Product: Topic 2: model, like, company, one, customer, product, data, said, time, say
- Apple: Topic 3: apple, image, video, new, feature, like, app, also, user, device
- Sustainability Tech: Topic 4: arm, pin, waste, meeting, rabbit, humane, recall, otter, greyparrot, recycling
- Google Gemini: Topic 5: google, gemini, search, feature, user, new, model, like, also, generative
- Enterprise AI: Topic 6: model, microsoft, openai, copilot, data, gpt, open, company, training, developer
- Tech Personalities: Topic 7: chromebook, velastegui, zuckerberg, supermaven, superannotate, fandom, wohl, cable, defcon, wendy
- Big Tech Company: Topic 8: openai, meta, chatgpt, company, also, safety, board, said, altman, content
- Content Platform: Topic 9: content, like, voice, company, one, people, said, user, news, use

## 2025년
- Online Communities: Topic 0: think, going, like, people, company, make, want, one, get, way
- Chatbot: Topic 1: model, openai, google, gemini, chatgpt, company, user, deepseek, like, new
- Language Models: Topic 2: model, company, grok, said, data, think, also, like, product, language
- Tech Personalities: Topic 3: company, google, openai, musk, said, billion, trump, startup, also, tech
- AI Search: Topic 4: data, google, perplexity, company, also, say, said, search, use, startup
- Hardware AI: Topic 5: nvidia, tesla, rtx, company, musk, blackwell, also, xai, chip, new
- Enterprise AI: Topic 6: microsoft, model, meta, company, data, openai, copilot, chatgpt, new, user
- AI Agents: Topic 7: openai, agent, company, alexa, amazon, said, new, like, system, also
- Apple: Topic 8: apple, feature, new, device, intelligence, also, company, model, game, humane
- Multimedia Tech: Topic 9: one, like, video, rtx, think, new, thing, google, want, get

In [1]:
import pandas as pd

# 연도별 토픽 레이블 딕셔너리
topic_label_mappings = {
    2021: {
        0: "Google Search",
        1: "Facial Recognition",
        2: "Customer Features",
        3: "Startup",
        4: "Voice Technology",
        5: "Platform",
        6: "Machine Learning",
        7: "Human-Technology Systems",
        8: "Social Media",
        9: "Programming"
    },
    2022: {
        0: "Robotics",
        1: "Data-Driven Analysis",
        2: "Language Models",
        3: "Startup",
        4: "Platform",
        5: "Supply Chain",
        6: "Human-AI Interaction",
        7: "Customer Service",
        8: "Generative AI",
        9: "Human-Technology Systems"
    },
    2023: {
        0: "EMS",
        1: "Chat GPT",
        2: "Generative AI",
        3: "Online Communities",
        4: "OpenAI Governance",
        5: "Google Search",
        6: "Search Engine",
        7: "Music Tools",
        8: "Startup",
        9: "Voice Technology"
    },
    2024: {
        0: "Online Communities",
        1: "Startup",
        2: "Product",
        3: "Apple",
        4: "Sustainability Tech",
        5: "Google Gemini",
        6: "Enterprise AI",
        7: "Tech Personalities",
        8: "Big Tech Company",
        9: "Content Platform"
    },
    2025: {
        0: "Online Communities",
        1: "Chatbot",
        2: "Language Models",
        3: "Tech Personalities",
        4: "AI Search",
        5: "Hardware AI",
        6: "Enterprise AI",
        7: "AI Agents",
        8: "Apple",
        9: "Multimedia Tech"
    }
}

# 대상 연도 리스트
years = [2021, 2022, 2023, 2024, 2025]

# 연도별 처리
for year in years:
    print(f"🔄 Processing year: {year}")

    # 1. 기사 데이터 로드 및 필터링
    article_df = pd.read_csv('data/01_combined_article.csv')
    article_df = article_df[article_df['date'] == year].reset_index(drop=True)

    # 2. 해당 연도 토픽 행렬 불러오기
    topic_matrix_path = f'data/result/02/02_article_{year}_topic_matrix.csv'
    topic_matrix = pd.read_csv(topic_matrix_path)

    # 3. 가장 높은 토픽 선택
    top_topic = topic_matrix.idxmax(axis=1)
    top_topic_num = top_topic.str.extract(r'(\d+)').astype(int)[0]

    # 4. 토픽 라벨 매핑
    topic_label_mapping = topic_label_mappings[year]
    article_df['topic_label'] = top_topic_num.map(topic_label_mapping)

    # 5. 저장
    save_path = f'data/result/02/02_article_{year}_labeled.csv'
    article_df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"✅ Saved labeled file for {year} to: {save_path}")

🔄 Processing year: 2021
✅ Saved labeled file for 2021 to: data/result/02/02_article_2021_labeled.csv
🔄 Processing year: 2022
✅ Saved labeled file for 2022 to: data/result/02/02_article_2022_labeled.csv
🔄 Processing year: 2023
✅ Saved labeled file for 2023 to: data/result/02/02_article_2023_labeled.csv
🔄 Processing year: 2024
✅ Saved labeled file for 2024 to: data/result/02/02_article_2024_labeled.csv
🔄 Processing year: 2025
✅ Saved labeled file for 2025 to: data/result/02/02_article_2025_labeled.csv


# 데이터 라벨링 작업 - Factor Analysis

## 2021년
- Factor_1: IS 발전 : NLP, GPT-3, 자동 완성, 코드 생성, 딥러닝, 로봇 설계 등 기술적 특성과 시스템 중심 AI 발전 내용을 다룸. 대표 문서들은 대부분 모델링, MLOps, 시스템 구조, 언어 처리 등 기술 구현 중심.
- Factor_2: IT 조직 : 조직 차원의 AI 활용, 물류 자동화, 모빌리티 생태계 기획, 기업 보안과 인수합병 등의 전략적 의사결정 이슈에 초점. 예: 물류 스타트업 성장, 정부-기업 간 교통계획, 사이버보안 기업 성장 전략.
- Factor_3: IT 시장 : 암호화폐 생태계, 스포츠 스트리밍, 프라이버시를 강조한 플랫폼, 광고 생태계 등 시장 및 산업 구조 변화와 관련된 주제. 예: Tether 논란, Apple 프라이버시 정책, Instagram 청소년 정책 등.
- Factor_4: IT 개인 : 개인의 행동, 인식, 감정과 관련된 기술의 영향 분석. 예: Babylon Health의 윤리 논란, 프라이버시 규제, Facebook 내부고발자 이슈 등 개인의 기술 수용성과 규제 반응 중심.
- Factor_5: IT 그룹 : 스타트업 생태계와 공동체, 투자자-창업자 네트워크, 기업 간 협업 생태계 등 집단 단위의 상호작용과 커뮤니티 확장 관련 주제. 예: 도시별 테크 허브 분석, 투자자 동향, 협업 기반 혁신 사례 등.

## 2022년
- Factor_1: IT 조직 : AI 기업의 마케팅 전략, 로보택시 상용화, 산업용 드론, 사이버 보안 등 ‘조직 수준의 AI 기술 적용, 제품화, 내부 역량 구축’에 초점. 기술을 조직 내부 역량과 연결짓거나 기업 전략의 일부로 다룸.
- Factor_2: IT 개인 : ChatGPT 사용성, 개인화 의료 플랫폼, 소비자 맞춤형 쇼핑, 언어 검색 판결 사례 등 ‘개인 경험과 반응, 인간-AI 상호작용’에 초점을 둔 문서가 다수. 특히 의료, 소비자 행동, 정책적 반응을 중심으로 개인의 기술 수용성 탐색.
- Factor_3: IS 발전 : ChatGPT 기술 자체, 언어모델 안전성, 방송·미디어 혁신 등 ‘AI 기술, 언어모델, 개인화 알고리즘의 진보와 응용’을 다룸. 기술 구조, 알고리즘, 시스템 설계와 같은 IS 기술 발전의 정체성과 부합.
- Factor_4: IT 시장 : 뉴스 미디어의 web3 전환, 메타의 다국어 번역, 언론의 혁신 부족 등 ‘AI와 사회, 산업, 미디어 시스템 간 관계 및 정책 영향’을 중심으로 산업/정책 구조와 시장 변화에 초점.
- Factor_5: IT 그룹 : 생성형 AI 이미지·영상 툴이 일반 사용자 커뮤니티와 창작자 그룹 내에서 생성·유통되며 만들어내는 집단적 창작 경험에 대한 논의. AI 기반 예술 도구의 확산이 협업과 집단 창작 생태계에 미치는 영향 강조.

## 2023년
- Factor_1: IT 조직 : Adobe, OpenAI, EvenUp, CoreWeave 등 기업 중심의 기술 개발과 전략적 실행을 다룬 기사들이 주를 이룸. 예: Firefly 생성형 모델 출시, 법률 자동화, 클라우드 인프라 전환 등 조직의 IT 활용과 기술 내재화 전략이 핵심.
- Factor_2: IT 시장 : 유럽연합 AI 법안, 캘리포니아 AI 규제 등 거버넌스와 시장 규칙 설정을 둘러싼 정책, 산업 동향 기사 다수. 이는 시장 구조 변화와 산업 간 상호작용이라는 'IT 시장'의 정의와 부합.
- Factor_3: IT 개인 : AI 기술의 사회적 수용, ChatGPT의 사용자 영향, API 활용 사례 등 개인의 행동·인지·경험을 중심으로 한 내용. 개인화된 상호작용 및 기술 수용성 문제를 다룸.
- Factor_4: IS 발전 : GPT-4, GPT-4 Turbo 등 멀티모달 언어모델의 기술적 진보, API 출시, 모델 성능 논의 등 시스템 아키텍처 및 기능성 중심의 기술 발전 기사 중심.
- Factor_5: IT 그룹 : Clearview AI, Squarespace, Harvard 교수, 음악 산업 CEO 등의 팟캐스트 및 대담 중심 기사로, 협업 커뮤니티, 윤리, 저작권, 창작 집단 등 다양한 집단의 역할과 상호작용을 조명.

## 2024년
- Factor_1: IT 조직 : Apple, Spotify, Amazon, Adobe 등 대형 기술 기업들이 자사 제품과 서비스에 AI 기능을 전략적으로 도입한 사례가 중심. 예: WWDC에서 발표한 Apple의 AI 계획, Adobe의 Firefly, Amazon과 Spotify의 AI 추천 시스템 도입 등은 조직 차원의 기술 내재화와 사업 전략으로 해석됨.
- Factor_2: IS 발전 : DevSecOps 트렌드, 수리 진단을 위한 음향 기반 AI, 신약 설계, 특허 분석 플랫폼 등은 모두 알고리즘·하드웨어·데이터 기반 시스템 혁신 사례. 이는 기술적 구조와 성능 향상이라는 전통적 IS 발전의 정의에 부합함.
- Factor_3: IT 시장 : Meta의 Llama 3, Mistral AI, xAI, AI2의 오픈모델 공개 등은 AI 모델 경쟁 구도 속에서의 시장 주도권 확보 전략. 오픈소스 생태계 형성과 기술 리더십 경쟁은 산업 및 플랫폼 시장의 역학을 반영함.
- Factor_4: IT 개인 : Google Gemini 기반 크롬 브라우저 기능, AI 멀티서치, 개인 맞춤형 기능 통합 등은 사용자의 웹 경험을 직접적으로 변화시키는 기술. 이는 개인의 수용성, 사용성, UX 변화 등 ‘IT와 개인의 상호작용’ 중심.
- Factor_5: IT 그룹 : 정치 딥페이크, 유명인 생성 콘텐츠 논란, 팬덤 집단의 대응, 아동 보호 관련 규제 등은 모두 집단적 대응, 사회 집합체 내 규범/신뢰/협력 구조와 관련. AI가 집단 간 상호작용 및 사회 질서에 미치는 영향 분석으로 분류됨.

## 2025년
- Factor_1: IS 발전 : Google Gemini, GPT-4.5, DeepSeek 등 주요 생성형 AI 모델과 기술 스택의 비교, 구조, 훈련 방식 등에 관한 기사 중심. 모델 성능, 기능 개선, AI 훈련 데이터 등 기술적 시스템 발전에 초점을 두고 있어 IS 발전으로 분류됨.
- Factor_2: IT 조직 : Meta의 저작권 소송, 기업 내부 결정 과정(예: Zuckerberg의 LLaMA 훈련 승인), Fiverr의 업무 자동화 전략, 아동 온라인 안전을 위한 조직 간 협업 등. 조직 수준에서의 의사결정, 전략 수립, 법적 대응이 중심.
- Factor_3: IT 시장 : 유럽의 오픈소스 LLM 전략, 공공 이익 기반 AI 생태계 구축, Gemma 3의 라이선스 문제, EU AI 법안 등 시장·산업 단위에서의 거버넌스, 경쟁, 규제, 생태계 구성 논의로 시장 관점에 적합.
- Factor_4: IT 개인 : AI 안전성 논쟁(AI doom), 정치 및 기술 규제에 대한 여론 반응, 대형 기업 인수에 따른 사용자 및 사회적 수용, 투자 심리 등 개인 수준에서 기술 변화에 대한 인지·행동·감정 반응 중심.
- Factor_5: IT 그룹 : Splice, Dow Jones, UiPath, Omi와 같은 다양한 산업 내 조직 리더와 기술 창업자들의 AI 도입·윤리·표현의 자유 등에 관한 인터뷰 중심. 이는 기업 내부 집단, 커뮤니티, 이해관계자 집단 간 상호작용과 신뢰 이슈를 반영하여 그룹 단위에 해당.




In [1]:
factor_label_mappings = {
    2021: {
        'Factor_1': 'IS 발전',
        'Factor_2': 'IT 조직',
        'Factor_3': 'IT 시장',
        'Factor_4': 'IT 개인',
        'Factor_5': 'IT 그룹'
    },
    2022: {
        'Factor_1': 'IT 조직',
        'Factor_2': 'IT 개인',
        'Factor_3': 'IS 발전',
        'Factor_4': 'IT 시장',
        'Factor_5': 'IT 그룹'
    },
    2023: {
        'Factor_1': 'IT 조직',
        'Factor_2': 'IT 시장',
        'Factor_3': 'IT 개인',
        'Factor_4': 'IS 발전',
        'Factor_5': 'IT 그룹'
    },
    2024: {
        'Factor_1': 'IT 조직',
        'Factor_2': 'IS 발전',
        'Factor_3': 'IT 시장',
        'Factor_4': 'IT 개인',
        'Factor_5': 'IT 그룹'
    },
    2025: {
        'Factor_1': 'IS 발전',
        'Factor_2': 'IT 조직',
        'Factor_3': 'IT 시장',
        'Factor_4': 'IT 개인',
        'Factor_5': 'IT 그룹'
    }
}

In [8]:
import pandas as pd

year = 2021

# 파일 경로
labeled_path = f'data/result/02/02_article_{year}_labeled.csv'
factor_scores_path = f'data/result/02/02_article{year}_factor_scores.csv'

# 데이터 불러오기
labeled_df = pd.read_csv(labeled_path)
factor_scores = pd.read_csv(factor_scores_path)

# 가장 높은 점수의 Factor 선택 및 라벨 매핑
factor_columns = ['Factor_1', 'Factor_2', 'Factor_3', 'Factor_4', 'Factor_5']
factor_scores['max_factor'] = factor_scores[factor_columns].idxmax(axis=1)
factor_scores['assigned_label'] = factor_scores['max_factor'].map(factor_label_mappings[year])  # ✅ 수정

# 병합
merged_df = pd.concat([
    labeled_df.reset_index(drop=True),
    factor_scores[['max_factor', 'assigned_label']].reset_index(drop=True)
], axis=1)

# 저장
merged_df.to_csv(f'data/result/02/02_article_{year}_merged_labeled.csv', index=False)
print(merged_df.head())

                                               title  date  \
0  Walmart is expanding its robot-powered fulfill...  2021   
1    Google is investigating another top AI ethicist  2021   
2  Amazon opens Alexa AI tech for the first time ...  2021   
3  FTC settles with photo storage app that pivote...  2021   
4                   Why Google’s union is a big deal  2021   

                                             content  \
0  Walmart is planning to increase the number of ...   
1  Google is investigating artificial intelligenc...   
2  Amazon will now allow third-party companies th...   
3  The Federal Trade Commission has reached a set...   
4  On January 4th, 2021, Google workers and contr...   

                                            keywords affiliations  \
0                    AI, Business, News, Robot, Tech        verge   
1                             AI, Google, News, Tech        verge   
2  AI, Amazon, Amazon Alexa, Business, Cars, News...        verge   
3             

In [9]:
import pandas as pd

# 병합할 파일들의 년도 리스트
years = [2021, 2022, 2023, 2024, 2025]

# 모든 파일을 담을 리스트
dfs = []

# 각 년도별 파일을 읽어서 리스트에 추가
for year in years:
    file_path = f'data/result/02/02_article_{year}_merged_labeled.csv'
    df = pd.read_csv(file_path)
    df['year'] = year  # 혹시 year 컬럼 없으면 추가
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
merged_df = pd.concat(dfs, ignore_index=True)

# 결과 저장
merged_df.to_csv('data/result/02/02_article_2021_2025_merged_labeled.csv', index=False)

print('✅ 저장 완료: data/result/02/02_article_2021_2025_merged_labeled.csv')

✅ 저장 완료: data/result/02/02_article_2021_2025_merged_labeled.csv
